## 微信爬蟲

### 取得列表頁

In [8]:
import requests

url = "https://weixin.sogou.com/weixin?type=2&ie=utf8&query=cctvnewscenter&tsn=0&ft=&et=&interation=&wxid=oIWsFt_IC706OXjJP2sn_T5MxVfs&usip=cctvnewscenter"
headers = {
"Host": "weixin.sogou.com",
"Referer": "https://weixin.sogou.com/weixin?type=2&s_from=input&query=cctvnewscenter&ie=utf8",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36"
}
res = requests.get(url, headers = headers)
res.encoding = 'utf-8'
#res.text

### 解析列表

In [9]:
import re
from bs4 import BeautifulSoup
doc=BeautifulSoup(res.text,'lxml')


def getURL(url):
    res=requests.get(url, allow_redirects=False)
    article_url=re.search(r'href="(.+?)"', res.text).group(1)
    return article_url

for news in doc.select('.news-list li'):
    fake_url = news.select_one('h3 a')['data-share']
    article_url = getURL(fake_url)
    
    data = {}
    data['url'] = article_url
    data['title'] = news.select_one('h3 a').text.strip()
    m = re.search(r'\d+', news.select_one('span.s2').text)
    if m:
        data['tm'] = re.search(r'\d+', news.select_one('span.s2').text).group(0)
    print(data)

{'url': 'http://mp.weixin.qq.com/s?src=11&timestamp=1568604330&ver=1855&signature=dPUfZIsfz5zseFqY7zKhyJQTitX9sDOQeiVn35hzpyzRu4fIkQ3A36Qt-fg6GmxA82g8k2LlMIVZQ*hm8RY1C67ItekplPxmvSfY9w4*ZIaa9M2EehtpYJbCviIm0alK&new=1', 'title': '中秋节,央视新闻送你一盒特别的月饼', 'tm': '1568360615'}
{'url': 'http://mp.weixin.qq.com/s?src=11&timestamp=1568604330&ver=1855&signature=dPUfZIsfz5zseFqY7zKhyJQTitX9sDOQeiVn35hzpyxQC1prHR2LCTCWr*jdn62dBi2Lw6g4lWhYfeJ0QRSohxpMe4JD3xxNqrfvS2sfLV2c2zPf7JIWnllpv5-Qw*SY&new=1', 'title': '央视新闻采访“光头刘sir”!他说最好的中秋礼物是……', 'tm': '1568421824'}
{'url': 'http://mp.weixin.qq.com/s?src=11&timestamp=1568604331&ver=1855&signature=dPUfZIsfz5zseFqY7zKhyJQTitX9sDOQeiVn35hzpyzYW47k6U5GsTLQMgT9RjwHrmb8PvvRWoXKh6v4FK*CY5R8bqP9v*9W4YunfBuVSOiYQNu9j3OYkXUepdH7xbhl&new=1', 'title': '央视发布2017年度十大国内新闻【附视频】', 'tm': '1514538803'}
{'url': 'http://mp.weixin.qq.com/s?src=11&timestamp=1568604331&ver=1855&signature=dPUfZIsfz5zseFqY7zKhyJQTitX9sDOQeiVn35hzpyzhXOALrMr8jVZrOslGt-PquX3VTv72Ykw1dnrx*50nCd4dwSB8IApe9

### 取得內文

In [11]:
def parseArticle(url):
    res = requests.get(url)
    soup =  BeautifulSoup(res.text,'lxml')
    content = ' '.join([s.text.strip() for s in soup.select('#js_content p')]).strip()
    return content

parseArticle('http://mp.weixin.qq.com/s?src=11&timestamp=1568604331&ver=1855&signature=dPUfZIsfz5zseFqY7zKhyJQTitX9sDOQeiVn35hzpyyCn22bsgEVobJAOCn8T4q92mDyT-GfBi*G-GYnki3Rl*g9ZpiophjloRzaWy4PT3lfE17i5hKz2cn2qrFuYzsV&new=1')

'中国的传统节日——中秋佳节 小编在此祝大家 中秋快乐！阖家欢聚！💕 “多姿多彩”的央视主播 也在此送上祝福啦！ 听说集齐这几种颜色 如果你觉得这三天假期太短 康辉说了 “再过十几天，就是国庆的7天长长长假了” 如果觉得7天还不够 “别急，再过一百多天，就过年啦” 所以呀，快乐就在于我们的这份心💕 今晚，就开开心心、快快乐乐地 邀君共品团圆酒 赏花赏月赏秋香吧！ 更多新闻 月亮到底是什么颜色的？这是一个科学问题…… 你画的月亮 是世界上最好看的月亮 中秋月不就是个圆吗？画圆有什么难的？不服来战！ 监制/徐冰 李浙\xa0 主编/张天宇 视频编辑/王茜 王义 张省 编辑/史萌 ©央视新闻'

### 完整爬蟲

In [12]:
ary = []
for news in doc.select('.news-list li'):
    fake_url = news.select_one('h3 a')['data-share']
    article_url = getURL(fake_url)
    
    data = {}
    data['url'] = article_url
    data['title'] = news.select_one('h3 a').text.strip()
    data['content'] = parseArticle(article_url)
    m = re.search(r'\d+', news.select_one('span.s2').text)
    if m:
        data['tm'] = re.search(r'\d+', news.select_one('span.s2').text).group(0)
    ary.append(data)

In [13]:
import pandas
df = pandas.DataFrame(ary)
df.head()

,content,title,tm,url
0,一盒月饼，九种口味 看图，一起品尝 更多新闻 中秋月不就是个圆吗？画圆有什么...,"中秋节,央视新闻送你一盒特别的月饼",1568360615,http://mp.weixin.qq.com/s?src=11&timestamp=156...
1,香港警务处机动部队警署警长 刘泽基 许多人亲切地称他为“光头刘sir” 7月30日 在葵涌一...,央视新闻采访“光头刘sir”!他说最好的中秋礼物是……,1568421824,http://mp.weixin.qq.com/s?src=11&timestamp=156...
2,央视发布2017年度十大国内新闻 点击查看↓↓↓ 中国共产党召开第十九次全国代表大会 ...,央视发布2017年度十大国内新闻【附视频】,1514538803,http://mp.weixin.qq.com/s?src=11&timestamp=156...
3,香港乱够了，市民受够了！8月17日下午，守护香港大联盟在金钟添马公园举行“反暴力、救香港”大...,央视快评 | 听一听香港民众的心声,1566042143,http://mp.weixin.qq.com/s?src=11&timestamp=156...
4,8月17日，47万多香港市民会聚金钟添马公园，在国歌声中发出共同呼吁：“反暴力、救香港！”酷...,"央视快评 | 寒蝉鸣处,乱港暴力已入穷途末路",1566133009,http://mp.weixin.qq.com/s?src=11&timestamp=156...


## Twitter 爬蟲

### 安裝 Python-Twitter
- ! pip install python-twitter

In [14]:
! pip install python-twitter

You are using pip version 9.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### 取得憑證

In [25]:
import twitter
api = twitter.Api(consumer_key='',
  consumer_secret='',
  access_token_key='',
  access_token_secret='')

### 抓取 Timeline 

In [16]:
user = 'realdonaldtrump'
statuses = api.GetUserTimeline(screen_name=user)
statuses[0].AsDict()


{'created_at': 'Mon Sep 16 00:31:58 +0000 2019',
 'favorite_count': 32498,
 'hashtags': [],
 'id': 1173394057072205825,
 'id_str': '1173394057072205825',
 'lang': 'en',
 'retweet_count': 9010,
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'text': '“We cannot have what happened to this President happen again.” Joe DiGenova.  “It is time for Justice to come barre… https://t.co/nrj7sHuSg7',
 'truncated': True,
 'urls': [{'expanded_url': 'https://twitter.com/i/web/status/1173394057072205825',
   'url': 'https://t.co/nrj7sHuSg7'}],
 'user': {'created_at': 'Wed Mar 18 13:46:38 +0000 2009',
  'description': '45th President of the United States of America🇺🇸',
  'favourites_count': 6,
  'followers_count': 64332591,
  'friends_count': 47,
  'geo_enabled': True,
  'id': 25073877,
  'id_str': '25073877',
  'listed_count': 108484,
  'location': 'Washington, DC',
  'name': 'Donald J. Trump',
  'profile_background_color': '6D5C18',
  'profile_backgr

In [17]:
user = 'realdonaldtrump'
statuses = api.GetUserTimeline(screen_name=user)
print([s.text for s in statuses])

['“We cannot have what happened to this President happen again.” Joe DiGenova.  “It is time for Justice to come barre… https://t.co/nrj7sHuSg7', 'The Fake News is saying that I am willing to meet with Iran, “No Conditions.” That is an incorrect statement (as usual!).', 'Here we go again with General Motors and the United Auto Workers. Get together and make a deal!', 'PLENTY OF OIL!', 'Saudi Arabia oil supply was attacked. There is reason to believe that we know the culprit, are locked and loaded de… https://t.co/KIcQ6LdQ90', '....sufficient to keep the markets well-supplied. I have also informed all appropriate agencies to expedite approva… https://t.co/3D2rYAkkiZ', 'Based on the attack on Saudi Arabia, which may have an impact on oil prices, I have authorized the release of oil f… https://t.co/dyskSEpgyH', '....Then, on top of it all, Kathleen Parker, of all people, wrote “In Case You Were Wondering, Trump Won The Debate… https://t.co/YgCiHc4HQe', 'Can’t believe the @washingtonpost wr

In [18]:
import pandas
ary = []
for s in statuses:
    ary.append(s.AsDict())
df= pandas.DataFrame(ary)
df.head()

,created_at,favorite_count,hashtags,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted_status,source,text,truncated,urls,user,user_mentions
0,Mon Sep 16 00:31:58 +0000 2019,32530.0,[],1173394057072205825,1173394057072205825,NaN,NaN,NaN,en,NaN,NaN,NaN,9017,NaN,"<a href=""http://twitter.com/download/iphone"" r...",“We cannot have what happened to this Presiden...,True,[{'expanded_url': 'https://twitter.com/i/web/s...,{'created_at': 'Wed Mar 18 13:46:38 +0000 2009...,[]
1,Sun Sep 15 23:02:16 +0000 2019,41890.0,[],1173371482812162048,1173371482812162048,NaN,NaN,NaN,en,NaN,NaN,NaN,11651,NaN,"<a href=""http://twitter.com/download/iphone"" r...",The Fake News is saying that I am willing to m...,NaN,[],{'created_at': 'Wed Mar 18 13:46:38 +0000 2009...,[]
2,Sun Sep 15 22:54:17 +0000 2019,33082.0,[],1173369474730381314,1173369474730381314,NaN,NaN,NaN,en,NaN,NaN,NaN,7415,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Here we go again with General Motors and the U...,NaN,[],{'created_at': 'Wed Mar 18 13:46:38 +0000 2009...,[]
3,Sun Sep 15 22:50:42 +0000 2019,59029.0,[],1173368574406250496,1173368574406250496,NaN,NaN,NaN,en,NaN,NaN,NaN,13114,NaN,"<a href=""http://twitter.com/download/iphone"" r...",PLENTY OF OIL!,NaN,[],{'created_at': 'Wed Mar 18 13:46:38 +0000 2009...,[]
4,Sun Sep 15 22:50:06 +0000 2019,48258.0,[],1173368423381962752,1173368423381962752,NaN,NaN,NaN,en,NaN,NaN,NaN,14418,NaN,"<a href=""http://twitter.com/download/iphone"" r...",Saudi Arabia oil supply was attacked. There is...,True,[{'expanded_url': 'https://twitter.com/i/web/s...,{'created_at': 'Wed Mar 18 13:46:38 +0000 2009...,[]


### 取得留言

In [21]:
import urllib.parse
tweet_id = 1171595113673347072
max_id = None
q = urllib.parse.urlencode({"q": "to:%s" % 'realDonaldTrump'})
replies = api.GetSearch(raw_query=q, since_id=tweet_id, max_id=max_id, count=100)

In [22]:
replies

[Status(ID=1173439049513746432, ScreenName=LjShaw3872, Created=Mon Sep 16 03:30:45 +0000 2019, Text='@realDonaldTrump #Iran https://t.co/JhAlpOW5BA'),
 Status(ID=1173439047617826816, ScreenName=keeponkeepngon, Created=Mon Sep 16 03:30:45 +0000 2019, Text='@realDonaldTrump @LouDobbs https://t.co/yvouRwN0G7'),
 Status(ID=1173439045080428544, ScreenName=URMoralsSUCK, Created=Mon Sep 16 03:30:44 +0000 2019, Text='@realDonaldTrump Does you base read these Tweets and think they are getting the inside scoop? Keep your phoney stor… https://t.co/P5xxlRW0lJ'),
 Status(ID=1173439044610600960, ScreenName=levinthauer, Created=Mon Sep 16 03:30:44 +0000 2019, Text="@realDonaldTrump @washingtonpost You don't read past headlines, do you?"),
 Status(ID=1173439044144910338, ScreenName=Datnurse, Created=Mon Sep 16 03:30:44 +0000 2019, Text='@realDonaldTrump @LouDobbs I wouldn’t say it’s whimpering , more like methodically crossing all the T’s and dotting all the I’s. Tick. Tock.'),
 Status(ID=117343904375

In [23]:
import pandas
ary = []
for r in replies:
    ary.append(r.AsDict())
df = pandas.DataFrame(ary)
df

,created_at,hashtags,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,media,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted_status,source,text,truncated,urls,user,user_mentions
0,Mon Sep 16 03:30:45 +0000 2019,[{'text': 'Iran'}],1173439049513746432,1173439049513746432,realDonaldTrump,1.173368e+18,25073877.0,und,"[{'display_url': 'pic.twitter.com/JhAlpOW5BA',...",NaN,NaN,NaN,NaN,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",@realDonaldTrump #Iran https://t.co/JhAlpOW5BA,NaN,[],{'created_at': 'Tue Jun 18 23:32:49 +0000 2013...,"[{'id': 25073877, 'id_str': '25073877', 'name'..."
1,Mon Sep 16 03:30:45 +0000 2019,[],1173439047617826816,1173439047617826816,realDonaldTrump,1.173394e+18,25073877.0,und,"[{'display_url': 'pic.twitter.com/yvouRwN0G7',...",NaN,NaN,NaN,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",@realDonaldTrump @LouDobbs https://t.co/yvouRw...,NaN,[],{'created_at': 'Fri Jan 27 05:49:44 +0000 2017...,"[{'id': 25073877, 'id_str': '25073877', 'name'..."
2,Mon Sep 16 03:30:44 +0000 2019,[],1173439045080428544,1173439045080428544,realDonaldTrump,1.173368e+18,25073877.0,en,NaN,NaN,NaN,NaN,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@realDonaldTrump Does you base read these Twee...,True,[{'expanded_url': 'https://twitter.com/i/web/s...,{'created_at': 'Mon Mar 23 00:55:13 +0000 2015...,"[{'id': 25073877, 'id_str': '25073877', 'name'..."
3,Mon Sep 16 03:30:44 +0000 2019,[],1173439044610600960,1173439044610600960,realDonaldTrump,1.173308e+18,25073877.0,en,NaN,NaN,NaN,NaN,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",@realDonaldTrump @washingtonpost You don't rea...,NaN,[],{'created_at': 'Wed Sep 22 19:27:21 +0000 2010...,"[{'id': 25073877, 'id_str': '25073877', 'name'..."
4,Mon Sep 16 03:30:44 +0000 2019,[],1173439044144910338,1173439044144910338,realDonaldTrump,1.173394e+18,25073877.0,en,NaN,NaN,NaN,NaN,NaN,NaN,"<a href=""http://twitter.com/#!/download/ipad"" ...",@realDonaldTrump @LouDobbs I wouldn’t say it’s...,NaN,[],{'created_at': 'Mon Apr 27 18:45:53 +0000 2009...,"[{'id': 25073877, 'id_str': '25073877', 'name'..."
5,Mon Sep 16 03:30:44 +0000 2019,[],1173439043755040768,1173439043755040768,realDonaldTrump,1.173369e+18,25073877.0,fr,NaN,NaN,NaN,NaN,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",@realDonaldTrump @POTUS Climate change!,NaN,[],{'created_at': 'Thu Feb 23 19:04:17 +0000 2012...,"[{'id': 25073877, 'id_str': '25073877', 'name'..."
6,Mon Sep 16 03:30:44 +0000 2019,[],1173439043444625408,1173439043444625408,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,67.0,{'created_at': 'Sun Sep 15 16:29:18 +0000 2019...,"<a href=""http://twitter.com/download/android"" ...",RT @JustInformU: @realDonaldTrump Isn't it iro...,NaN,[],{'created_at': 'Tue Mar 31 04:09:07 +0000 2009...,"[{'id': 818517696077774849, 'id_str': '8185176..."
7,Mon Sep 16 03:30:43 +0000 2019,[],1173439040164696064,1173439040164696064,NaN,NaN,NaN,en,NaN,NaN,NaN,NaN,352.0,{'created_at': 'Sun Sep 15 23:42:28 +0000 2019...,"<a href=""http://twitter.com/#!/download/ipad"" ...",RT @TrueEyeTheSpy: @realDonaldTrump As the Lef...,NaN,[],{'created_at': 'Fri Apr 10 23:02:10 +0000 2009...,"[{'id': 1000739286206107649, 'id_str': '100073..."
8,Mon Sep 16 03:30:42 +0000 2019,[],1173439037899558912,1173439037899558912,NaN,NaN,NaN,en,"[{'display_url': 'pic.twitter.com/7KDuYX6t7p',...",NaN,NaN,NaN,2.0,{'created_at': 'Mon Sep 16 01:29:19 +0000 2019...,"<a href=""http://twitter.com/#!/download/ipad"" ...",RT @FreeGirlNowNYC: @realDonaldTrump Calm down...,NaN,[],{'created_at': 'Sat Mar 05 19:20:15 +0000 2011...,"[{'id': 1272776588, 'id_str': '1272776588', 'n..."
9,Mon Sep 16 03:30:42 +0000 2019,[],1173439036481925120,1173439036481925120,realDonaldTrump,1.173368e+18,25073877.0,en,NaN,NaN,NaN,NaN,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",@realDonaldTrump Is the king of Saudi Arabia y...,NaN,[],{'created_at': 'Wed Feb 25 13:32:01 +0000 2009...,"

### 取得轉推

In [24]:
api.GetRetweets(statusid='1171595113673347072')

[Status(ID=1173397561375223809, ScreenName=johnwatford94, Created=Mon Sep 16 00:45:53 +0000 2019, Text='RT @realDonaldTrump: Senator Ben Sasse has done a wonderful job representing the people of Nebraska. He is great with our Vets, the Militar…'),
 Status(ID=1173395326167867392, ScreenName=c11blb, Created=Mon Sep 16 00:37:01 +0000 2019, Text='RT @realDonaldTrump: Senator Ben Sasse has done a wonderful job representing the people of Nebraska. He is great with our Vets, the Militar…'),
 Status(ID=1173386668159750145, ScreenName=ca_skysunfun, Created=Mon Sep 16 00:02:36 +0000 2019, Text='RT @realDonaldTrump: Senator Ben Sasse has done a wonderful job representing the people of Nebraska. He is great with our Vets, the Militar…'),
 Status(ID=1173384515370323969, ScreenName=PaulStr76514228, Created=Sun Sep 15 23:54:03 +0000 2019, Text='RT @realDonaldTrump: Senator Ben Sasse has done a wonderful job representing the people of Nebraska. He is great with our Vets, the Militar…'),
 Status(ID=117